<a href="https://colab.research.google.com/github/Lexie-D/Breast-Cancer-Classification/blob/main/Breast_Cancer_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>